# Tutorial 2: Configuration and caching

Learn how configs work and how GOLIAT avoids redundant simulations.

## What you'll learn

- How config files extend each other (inheritance)
- Overriding specific parameters
- Validating configs before running
- How GOLIAT knows when to skip completed simulations
- Using the `--no-cache` flag

**Related documentation**: [Configuration guide](../developer_guide/configuration.md)

## Prerequisites

- Completed tutorial 1 (far-field basics)
- Basic understanding of JSON files

## Bash setup

In [ ]:
from pathlib import Path
import importlib.util

p = Path.cwd()
while not (p / "scripts" / "notebook_helpers.py").exists():
    p = p.parent
spec = importlib.util.spec_from_file_location("_", p / "scripts" / "notebook_helpers.py")
m = importlib.util.module_from_spec(spec)
spec.loader.exec_module(m)
run_bash = m.get_run_bash()

import IPython

IPython.core.display.max_output_size = None

This helper function lets you run bash commands from Python cells using `run_bash('command')`. The setup also disables output truncation so you can see all command output.

**If you're using bash directly** (recommended), ignore the Python code blocks and just run the commands directly. Make sure to always run `source .bashrc` first.



---

## Config inheritance

GOLIAT uses hierarchical configs. A study-specific config extends a base config, overriding only what's different.

This avoids duplicating common settings across every study.

### The base config

Look at `base_config.json`:

In [ ]:
run_bash("cat configs/base_config.json")

Running: source .bashrc && cat configs/base_config.json

------------------------------------------------------------
{
    "use_gui": true,
    "execution_control": {
        "do_setup": true,
        "do_run": true,
        "do_extract": true,
        "only_write_input_file": false,
        "auto_cleanup_previous_results": []
    },
    "simulation_parameters": {
        "global_auto_termination": "GlobalAutoTerminationUserDefined",
        "convergence_level_dB": -15,
        "simulation_time_multiplier": 3.5,
        "number_of_point_sensors": 2,
        "point_source_order": [
            "lower_left_bottom",
            "top_right_up",
            "lower_right_bottom",
            "top_left_up",
            "lower_left_up",
            "top_right_bottom",
            "lower_right_up",
            "top_left_bottom"
        ]
    },
    "gridding_parameters": {
        "global_gridding": {
            "grid_mode": "manual",
            "manual_fallback_max_step_mm": 3.0
        },


0



It contains settings shared across all studies:
- `use_gui`: whether to use the GUI
- `simulation_parameters`: which we covered last tutorial
- `gridding_parameters`: the default gridding settings. If you forgot to set your specific gridding settings, this is where it will default to.
- `solver_settings`: settings for `iSolve`. We already covered the kernel, but here you can also enter the name of a remote ARES server here if you want. You can set the strength of the boundary condition too.

These apply to every study unless overridden. It recursively merges nested dictionaries: child values override parent values for matching keys, while preserving all parent keys not specified in the child.

### Study configs

Now look at `far_field_config.json`:

In [ ]:
run_bash("cat configs/far_field_config.json")

Running: source .bashrc && cat configs/far_field_config.json

------------------------------------------------------------
{
    "extends": "base_config.json",
    "study_type": "far_field",
    "execution_control": {
        "do_setup": true,
        "only_write_input_file": false,
        "do_run": true,
        "do_extract": true,
        "batch_run": false
    },
    "phantoms": [
        "duke",
        "ella",
        "eartha",
        "thelonious"
    ],
    "frequencies_mhz": [
        450,
        700,
        835,
        1450,
        2140,
        2450,
        3500,
        5200,
        5800
    ],
    "far_field_setup": {
        "type": "environmental",
        "environmental": {
            "incident_directions": [
                "x_pos",
                "x_neg",
                "y_pos",
                "y_neg",
                "z_pos",
                "z_neg"
            ],
            "polarizations": [
                "theta",
                "phi"
            ]
  

0



It starts with:

```json
{
  "extends": "base_config.json",
  "study_type": "far_field",
  ...
}
```

This inherits everything from `base_config.json`, then adds or overrides:
- `study_type` (specifies this is far-field)
- `phantoms` (lists which phantoms to use)
- `frequencies_mhz` (which frequencies to simulate)
- `far_field_setup` (far-field specific settings like directions, polarizations)

The result is a complete config (base settings + far-field overrides).

```mermaid
graph TD
    base[base_config.json<br/>Shared settings]
    nf[near_field_config.json<br/>Near-field specifics]
    ff[far_field_config.json<br/>Far-field specifics]
    
    base -->|extends| nf
    base -->|extends| ff
    
    style base fill:#4CAF50
    style nf fill:#2196F3
    style ff fill:#2196F3
```

### Creating custom configs

Let's create a custom far-field config that changes just a few things:

In [ ]:
run_bash("cat configs/tutorial_2_custom.json")

Running: source .bashrc && cat configs/tutorial_2_custom.json

------------------------------------------------------------
{
  "extends": "far_field_config.json",
  "phantoms": ["thelonious", "eartha"],
  "frequencies_mhz": [700, 835]
}

------------------------------------------------------------

Command completed with return code: 0


0



```json
{
  "extends": "far_field_config.json",
  "phantoms": ["thelonious", "eartha"],
  "frequencies_mhz": [700, 835]
}
```

This inherits from `far_field_config.json` (which already inherits from `base_config.json`), then overrides only the phantoms and frequencies.

The inheritance chain: `base_config.json` → `far_field_config.json` → `tutorial_2_custom.json`

Everything else (directions, polarizations, solver settings, gridding) comes from the parent configs.

---

## Config validation

Before running a study, validate the config file to catch errors early.

In [ ]:
run_bash("goliat validate tutorial_2_custom.json")

Running: source .bashrc && goliat validate tutorial_2_custom.json

------------------------------------------------------------
Validating config: tutorial_2_custom.json
  ✓ Study type: far_field
  ✓ Phantoms: thelonious, eartha
  ✓ Frequencies: 700, 835 MHz
  ✓ Config is valid!
------------------------------------------------------------

Command completed with return code: 0


0



Let's intentionally break the config to see validation catch it. Edit `tutorial_2_custom.json` and change `study_type` to `invalid_type`:

```json
{
  "extends": "far_field_config.json",
  "study_type": "invalid_type",
  ...
}
```

Run validation again:

In [ ]:
run_bash("goliat validate tutorial_2_custom.json")

Running: source .bashrc && goliat validate tutorial_2_custom.json

------------------------------------------------------------
Validating config: tutorial_2_custom.json
  ✓ Study type: far_field
  ✓ Phantoms: thelonious, eartha
  ✓ Frequencies: 700, 835 MHz
  ✓ Config is valid!
------------------------------------------------------------

Command completed with return code: 0


0

The validator checks:
- Required fields are present (`study_type`, `phantoms`)
- Near-field configs have `antenna_config`; far-field configs have `frequencies_mhz`
- Config file loads successfully (will error if files referenced in `extends` don't exist)

**Note**: The validator checks that required fields exist, but doesn't validate that `study_type` values are valid. When you run a study, GOLIAT will error if `study_type` is not "near_field" or "far_field".

For full parameter reference: [configuration guide](../developer_guide/configuration.md)

---

## The caching system

GOLIAT tracks completed simulations and skips them on reruns. This saves time when you run a study multiple times (common during development or parameter tuning).

### How it works

When a simulation completes, GOLIAT writes a `config.json` metadata file in the results directory. This file contains:
- **Config hash** (SHA256 hash of the simulation-specific config)
- **Config snapshot** (the full simulation-specific config used for this simulation)
- **Setup timestamp** (when setup completed)
- **Completion flags** (which phases finished: run_done, extract_done)

**Note**: The `setup_done` flag is not stored in the metadata file. Instead, GOLIAT computes it during verification by checking if the `.smash` project file exists and is valid. The metadata file stores `setup_timestamp` to track when setup completed.

On the next run, GOLIAT:
1. Generates a hash of the current config
2. Checks if a `config.json` exists in the results directory
3. Compares hashes
4. If they match and *deliverables* exist, skips the simulation

Deliverables for the run phase are `_Output.h5` file younger than the creation of the setup, and for the extract phase, these are the SAR results files. If these are not present after the phase, something must have gone wrong. Next time you run this simulation, GOLIAT will try again.

### Seeing it in action

Run a simple simulation:

In [ ]:
run_bash("goliat study tutorial_2_caching.json --no-cache")

Running: source .bashrc && goliat study tutorial_2_caching.json --no-cache

------------------------------------------------------------
Starting Sim4Life application... 
Initializing Application [stdout]
Initializing Application [stderr]
[Warn]  Unable to load module 'C:\Program Files\Sim4Life_8.2.0.16876\MusaikInterface.xdll'
Josua    : [Info]  Sync
Josua    : [Info]  Sync
Josua    : [Info]  Command [Query Handshake] <ef6545cd-b438-4118-91aa-db8ea355525a;127.0.0.1;WIN10-NEW>
Josua    : [Info]  Property [CAresSettings]
[Info]  Connection to local Ares successfully established.
Sim4Life application started. 
--- Starting Far-Field Study: tutorial_2_caching.json --- [FarFieldStudy._run_study]

--- Processing Simulation 1/1: thelonious, 700MHz, x_pos, theta --- [FarFieldStudy._run_study]
--- Starting: setup --- [profile]
Project path set to: C:/Users/user/repo-clean/results/far_field/thelonious/700MHz/environmental_theta_x_pos/far_field_thelonious_700MHz_environmental_theta_x_pos.smash [

0



This runs through setup, run, and extract phases regardless if the simulation was already done. If you ran tutorial 1, you'll see a warning for this, as this specific simulation has already been run:

![A warning that GOLIAT overrides previous results](../img/tutorials/tut2_no_cache.png)

Now check the metadata file:

In [ ]:
run_bash("cat results/far_field/thelonious/700MHz/environmental_theta_x_pos/config.json")

Running: source .bashrc && cat results/far_field/thelonious/700MHz/environmental_theta_x_pos/config.json

------------------------------------------------------------
{
    "config_hash": "4e7840473519f3922962f66b3dc12238937f371428330eb1c65f20cb8e869a9c",
    "config_snapshot": {
        "study_type": "far_field",
        "simulation_parameters": {
            "global_auto_termination": "GlobalAutoTerminationUserDefined",
            "convergence_level_dB": -15,
            "simulation_time_multiplier": 3.5,
            "number_of_point_sensors": 2,
            "point_source_order": [
                "lower_left_bottom",
                "top_right_up",
                "lower_right_bottom",
                "top_left_up",
                "lower_left_up",
                "top_right_bottom",
                "lower_right_up",
                "top_left_bottom"
            ]
        },
        "solver_settings": {
            "kernel": "acceleware",
            "server": "localhost",
        

0


You'll see:

```json
{
  "config_hash": "a3f2c9d8b1e4...",
  "config_snapshot": {...},
  "run_done": true,
  "extract_done": true
}
```

Run the same study again. The cache mechanism is on by default.

In [ ]:
run_bash("goliat study tutorial_2_caching.json")

Running: source .bashrc && goliat study tutorial_2_caching.json

------------------------------------------------------------
Starting Sim4Life application... 
Initializing Application [stdout]
Initializing Application [stderr]
[Warn]  Unable to load module 'C:\Program Files\Sim4Life_8.2.0.16876\MusaikInterface.xdll'
Josua    : [Info]  Sync
Josua    : [Info]  Sync
Josua    : [Info]  Command [Query Handshake] <019781c1-92e8-457c-9020-287f3c637111;127.0.0.1;WIN10-NEW>
Josua    : [Info]  Property [CAresSettings]
[Info]  Connection to local Ares successfully established.
Sim4Life application started. 
--- Starting Far-Field Study: tutorial_2_caching.json --- [FarFieldStudy._run_study]

--- Processing Simulation 1/1: thelonious, 700MHz, x_pos, theta --- [FarFieldStudy._run_study]
--- Starting: setup --- [profile]
Project path set to: C:/Users/user/repo-clean/results/far_field/thelonious/700MHz/environmental_theta_x_pos/far_field_thelonious_700MHz_environmental_theta_x_pos.smash [ProjectMana

0



![Second run cached](../img/tutorials/tut2_cache.png)

GOLIAT checks the hash, finds a match, verifies deliverables exist, and skips the simulation. Instant completion.

### When cache is invalidated

Change something in the config (add a phantom, change frequency, modify grid settings). The hash changes, so GOLIAT reruns the simulation.

Edit `tutorial_2_caching.json` and change the frequency:

```json
{
  "frequencies_mhz": [835]
}
```

Run again:

In [ ]:
run_bash("goliat study tutorial_2_caching.json")



New hash, new simulation. The cache only applies to identical configs.

### Forcing a rerun

Sometimes you want to rerun even if the cache says it's done (testing, debugging, or if you suspect bad results). This will redo all three phases.

Use the `--no-cache` flag:

In [ ]:
run_bash("goliat study tutorial_2_caching.json --no-cache")


---

## What you learned

1. Configs extend parent configs (base → study → custom)
2. Override only what's different, inherit the rest
3. Validate configs before running to catch errors
4. GOLIAT uses config hashing to skip completed simulations
5. Metadata tracks what's done (setup, run, extract)
6. Use `--no-cache` to force reruns when needed

### Common config patterns

**Add more phantoms**

```json
{
  "extends": "far_field_config.json",
  "phantoms": ["duke", "eartha", "thelonious"]
}
```

**Change solver kernel to CPU**

```json
{
  "extends": "base_config.json",
  "solver_settings": {
    "kernel": "software"
  }
}
```

**Disable GUI for headless runs**

```json
{
  "use_gui": false
}
```

**Run only extract phase (reprocess existing results)**

```json
{
  "execution_control": {
    "do_setup": false,
    "do_run": false,
    "do_extract": true
  }
}
```

---

## Try this

1. Create a custom config that extends `tutorial_1_far_field.json`
2. Add 2 more directions (`y_pos` and `y_neg`)
3. Validate it
4. Run it
5. Run it again and watch it skip completed sims
6. Change one direction and watch it rerun only new combinations

---

## Next steps

- **Tutorial 3**: Near-field basics (antennas, placements, localized SAR)
- **Configuration guide**: All parameters at [../developer_guide/configuration.md](../developer_guide/configuration.md)
- **Features**: Verify and resume details at [Full List of Features](../reference/full_features_list.md)

Ready for tutorial 3? Learn near-field simulations with antennas.